<a href="https://colab.research.google.com/github/pspuser/Urban_Python/blob/main/Chistiakov_VV_PythonUrban_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [94]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify  # mapclassify для визуализации
!pip install osmnx # устанавливаем модуль osmnx

In [95]:
# TODO собрать импорты всех модулей в одном месте
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import json
import requests

In [97]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Красносельский район, Санкт-Петербург"  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/pspuser/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/pspuser/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [98]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf(TERRITORY_NAME)
district.explore() # TODO убрать explore()

## Улицы

### Загрузка файла с улицами (Extract)

In [99]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf = gpd.read_file(STREETS_FILE_URL, mask=district)
gdf.info()
gdf

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      1582 non-null   object  
 1   geometry  1582 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 24.8+ KB


,name,geometry
0,улица Доблести,"LINESTRING (3359594.246 8368940.704, 3359610.6..."
1,проспект Будённого,"LINESTRING (3349643.598 8367056.893, 3349646.9..."
2,улица Стойкости,"LINESTRING (3360780.923 8361696.899, 3360753.1..."
3,улица Солдата Корзуна,"LINESTRING (3362732.566 8364821.874, 3362740.1..."
4,Восточный переулок,"LINESTRING (3350559.957 8338315.414, 3350805.8..."
...,...,...
1577,Лиговский путепровод,"LINESTRING (3360027.836 8360383.607, 3360102.8..."
1578,Таллинское шоссе,"LINESTRING (3358251.878 8357338.717, 3358302.2..."
1579,Таллинское шоссе,"LINESTRING (3358029.339 8357012.149, 3358023.5..."
1580,Таллинское шоссе,"LINESTRING (3358052.750 8357050.952, 3358029.3..."


### Обработка данных с улицами (Transform)

In [101]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "name": "Название"
}
gdf.rename(columns=rename_columns, inplace=True)

# Поменял местами 2 задания, потому что после gdf.dissolve(), судя
# по геодатафрейму и его инфо, из двух столбцов остается один

# TODO сгруппировать и объединить геометрии с одинаковыми названиями

gdf = gdf.dissolve(by="Название")
gdf

,geometry
Название,
1-й Купальный переулок,"LINESTRING (3350719.422 8338033.644, 3350939.5..."
1-й Петергофский мост,"MULTILINESTRING ((3357212.599 8365672.439, 335..."
1-й проезд,"MULTILINESTRING ((3354027.771 8348984.834, 335..."
1-й проезд Красной улицы,"LINESTRING (3351597.076 8361086.489, 3351646.1..."
1-я Берёзовая улица,"LINESTRING (3351006.360 8368299.479, 3350930.0..."
...,...
улица Чекистов,"MULTILINESTRING ((3354082.718 8365335.330, 335..."
улица Щорса,"LINESTRING (3348360.796 8361203.960, 3348452.2..."
улица Энгельса,"LINESTRING (3350739.894 8362337.506, 3350827.1..."


### Сохранение слоя с улицами (Load)

In [102]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [103]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf = gpd.read_file(KGIOP_FILE_URL, mask=district)
gdf

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,2896,—,"Сосновая поляна. Парк, остатки исторической пл...",1710-1714,Ветеранов пр.; Летчика Пилютова ул.; Балтийска...,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3354589.144 8362088.243)
1,2897,—,Мемориал в память обороны города в 1941-1944 г...,1966,"Маршала Жукова пр. (напротив дома 38, корп.1 п...",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Ансамбль,POINT (3360642.587 8363693.884)
2,2898,—,Мемориал в память обороны города в 1941-1944 г...,1968,"Петергофское шоссе, 14-й км (на пересечении с ...",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3359555.997 8365524.138)
3,2899,—,Мемориал в память обороны города в 1941-1944 г...,1944,"Петергофское шоссе, 16-км (около Тамбасова ул.)",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3357077.023 8365562.254)
4,2900,—,Матисов канал,1800-около,"Петергофское шоссе, в створе 2-й Комсомольской...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3355630.204 8367035.215)
...,...,...,...,...,...,...,...,...,...
102,3006,—,"Братская могила моряков крейсера ""Аврора"", пог...",1964 (памятник); 1984 (мемориал),"пос. Можайский, на территории Нагорного парка",объект культурного наследия регионального знач...,Решение исполкома Ленгорсовета № 124 от 27.02....,Памятник,POINT (3353358.507 8331564.745)
103,3007,—,"Братская могила воинов Советской Армии, погибш...",—,пос. Урицк,объект культурного наследия регионального знач...,Решение Исполкома Ленгорсовета от 03.05.1976 г...,Памятник,POINT (3358341.835 8362845.606)
104,3008,—,Исторические трассы Петергофской и Нарвской дорог,17 в.-18 в.сер.; 1773-1784 (реконструирована),"пр. Стачек и Петергофское шоссе, пр. Маршала Ж...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3359992.147 8360308.446)
105,7363,Сооружения периода 1941-1943 гг.,ДОТ артиллерийский,1941-1943,"Красное Село, ул. Нарвская, д. 8, к. 1",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-192 от 07.05.2015,Памятник,POINT (3347383.879 8337529.501)


### Обработка объектов культурного наследия (Transform)

In [104]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
gdf["lon"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).x
gdf["lat"] = gdf.to_crs(3857).geometry.centroid.to_crs(4326).y

### Сохранение слоя с объектами культурного наследия (Load)

In [92]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "id": "Идентификатор",
    "ensemble_name": "Наименование ансамбля",
    "object_name": "Наименование объекта",
    "occurrence_time": "Время возникновения",
    "object_location": "Местоположение объекта",
    "historical_category": "Историческая категория",
    "normative_act": "Нормативный акт",
    "object_type": "Тип объекта",
    "lon": "Долгота",
    "lat": "Широта"
}
gdf.rename(columns=rename_columns, inplace=True)
gdf.head()

,Идентификатор,Наименование ансамбля,Наименование объекта,Время возникновения,Местоположение объекта,Историческая категория,Нормативный акт,Тип объекта,geometry,Долгота,Широта
0,2896,—,"Сосновая поляна. Парк, остатки исторической пл...",1710-1714,Ветеранов пр.; Летчика Пилютова ул.; Балтийска...,объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Ансамбль,POINT (3354589.144 8362088.243),30.134787,59.830461
1,2897,—,Мемориал в память обороны города в 1941-1944 г...,1966,"Маршала Жукова пр. (напротив дома 38, корп.1 п...",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Ансамбль,POINT (3360642.587 8363693.884),30.189166,59.837709
2,2898,—,Мемориал в память обороны города в 1941-1944 г...,1968,"Петергофское шоссе, 14-й км (на пересечении с ...",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3359555.997 8365524.138),30.179405,59.845969
3,2899,—,Мемориал в память обороны города в 1941-1944 г...,1944,"Петергофское шоссе, 16-км (около Тамбасова ул.)",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3357077.023 8365562.254),30.157136,59.846141
4,2900,—,Матисов канал,1800-около,"Петергофское шоссе, в створе 2-й Комсомольской...",объект культурного наследия регионального знач...,Распоряжение мэра Санкт-Петербурга № 108-р от ...,Памятник,POINT (3355630.204 8367035.215),30.144139,59.852787


In [105]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf.to_crs(4326).to_file('KGIOP.geojson', driver='GeoJSON')